<a href="https://colab.research.google.com/github/Shashankwer/Tensorflow_Testing/blob/master/tf_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In Tensorflow 2,eager execution is turned on by default. The user interface is intutive and flexible(running one of operation is much easier and faster). This can come at the expense of performance and deployability



In [2]:
import tensorflow as tf

In [3]:
import traceback
import contextlib

In [4]:
@contextlib.contextmanager
def assert_raises(error_class):
  try:
    yield
  except error_class as e:
    print('Caught excepted exception \n {}'.format(error_class))
    traceback.print_exc(limit=2)
  except Exception as e:
    raise e
  else:
    raise Exception('Expected {} to be raised but it was not raise'.format(error_class))


In [5]:
@tf.function
def add(a,b):
  return a+b

add(tf.ones([2,2]),tf.ones([2,2]))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [6]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
  result = add(v,1.0)
tape.gradient(result,v)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [7]:
@tf.function
def dense_layer(x,w,b):
  return add(tf.matmul(x,w),b)

dense_layer(tf.ones([3,2]),tf.ones([2,2]),tf.ones([2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

In [8]:
tf.matmul(tf.ones([3,2]),tf.ones([2,3]))

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.]], dtype=float32)>

In [9]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100,3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1,200,200,100])
conv_layer(image);
conv_fn(image)
print("Eager conv :",timeit.timeit(lambda: conv_layer(image),number=100))
print("Function conv:",timeit.timeit(lambda: conv_fn(image),number=100))

Eager conv : 14.921743999000228
Function conv: 14.537456206000115


## Tracing: 


Python's dynamic typing means that one can call functions with a variety of argument types and a python can do something in each scenario. 

Yet to create Tensorflow Graph, static `dtypes` and shape dimension require `tf.function` bridges this gaps by wrapping a Python function to create a `Function` object. Based on the given inputs, the `Function` selects the appropriate graph for the given inputs retracing the python function as necessary. 

Once we understand why and when the tracing happends, it is much easier to use tf.function effectively. 



In [10]:
@tf.function
def double(a):
  print("Tracing with ",a)
  return a+a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()

Tracing with  Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with  Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with  Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



In [11]:
print(double(tf.constant("b")))

tf.Tensor(b'bb', shape=(), dtype=string)


In [12]:
print(double.pretty_printed_concrete_signatures())

double(a)
  Args:
    a: int32 Tensor, shape=()
  Returns:
    int32 Tensor, shape=()

double(a)
  Args:
    a: string Tensor, shape=()
  Returns:
    string Tensor, shape=()

double(a)
  Args:
    a: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()


The `tf.function` creates a chaced, dynamic dispatch layer over TensorFLow's graph tracing logic. TO be more specific about the terminology: 

1. A tf.Graph is the raw, language agnostic representation of the computation
2. A concreteFunction is eagerly wrapped around a tf.Graph
3. A function manages to cache ConcreteFunction and picks the right one for the inputs
4. tf.function wraps a Python function and returns a python object

In [13]:
double_strings = double.get_concrete_function(tf.constant("a"))
print("Excuting traces")
print(double_strings(tf.constant("a")))
print(double_strings(tf.constant("b")))


Excuting traces
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)


In [14]:
double_strings_from_inputspec = double.get_concrete_function(tf.TensorSpec(shape=[],dtype=tf.string))
print(double_strings_from_inputspec(tf.constant("c")))

Tracing with  Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'cc', shape=(), dtype=string)


In [15]:
print(double_strings)

ConcreteFunction double(a)
  Args:
    a: string Tensor, shape=()
  Returns:
    string Tensor, shape=()


In [16]:
print(double_strings.structured_input_signature)
print(double_strings.structured_outputs)

((TensorSpec(shape=(), dtype=tf.string, name='a'),), {})
Tensor("Identity:0", shape=(), dtype=string)


In [17]:
with assert_raises(tf.errors.InvalidArgumentError):
  double_strings(tf.constant(1))

Caught excepted exception 
 <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>


Traceback (most recent call last):
  File "<ipython-input-4-46ae65889eac>", line 4, in assert_raises
    yield
  File "<ipython-input-17-e4e2860a4364>", line 2, in <module>
    double_strings(tf.constant(1))
tensorflow.python.framework.errors_impl.InvalidArgumentError: cannot compute __inference_double_625 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_625]


In [18]:
@tf.function
def pow(a,b):
  return a**b

square = pow.get_concrete_function(a=tf.TensorSpec(None,tf.float32),b=2)
print(square)

ConcreteFunction pow(a, b=2)
  Args:
    a: float32 Tensor, shape=<unknown>
  Returns:
    float32 Tensor, shape=<unknown>


In [19]:
assert square(tf.constant(10.0))==100

with assert_raises(TypeError):
  square(tf.constant(10.0),b=3)

Caught excepted exception 
 <class 'TypeError'>


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/function.py", line 1669, in _call_impl
    cancellation_manager)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/function.py", line 1714, in _call_with_flat_signature
    self._flat_signature_summary(), ", ".join(sorted(kwargs))))
TypeError: pow(a) got unexpected keyword arguments: b.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-4-46ae65889eac>", line 4, in assert_raises
    yield
  File "<ipython-input-19-77f1457582f0>", line 4, in <module>
    square(tf.constant(10.0),b=3)
TypeError: ConcreteFunction pow(a, b) was constructed with int value 2 in b, but was called with int value 3


In [20]:
#Obtaining graphs

graph  = double_strings.graph
for node in graph.as_graph_def().node:
  print(f'{node.input}->{node.name}')

[]->a
['a', 'a']->add
['add']->Identity


# Debugging

In general, debugging code is easier in eager mode than inside `tf.function`. One should ensure that the code executes error free in eager mode befroe decorating with `tf.function`. To assist in the debugging process, one can call `tf.config.run_functions_eagerly(True)` to globally disable and reenable `tf.function`

* Plain old print calls are executed when the tracking down the function is retraced 
* tf.print will execute every time
* tf.debugging.enable_check_numerics is an easy way to track down where NaNs and Inf are created. 
* `pdb` can help one to understand whats going on during tracing


# Tracing Semantics. 

## Cache rules: 

A `Function` determines whether to resue a traced concrete function by computing cache key from inputs arg and kwags

* A key generated is in the form of tf.Tensor argument is its shape and dtype
* Starting tensorflow 2.3, the key generated for a tf.Varaible argument is its id()
* The key for a Python primitive is its value. The key generated for nested dicts, list and named tuples and attrs is the flattened tuple. 
* For all other Python types, the keys are based on the object id() so that the methods are traced independently from each other 

## Controlling retracing: 

Retracing helps to ensure that TensorFLow generates correct graphs for each of the inputs. However tracing is an expensive operation!. If ine Function retraces a new graph for eery call this would cause the function to trace more slowly than making use of `tf.function`

To control the tracing behavior, one can use the following techniques

* Specify the input signature in `tf.function` to limit tracing 


In [21]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None],dtype=tf.int32),))
def next_collatz(x):
  print("Tracing with",x)
  return tf.where(x%2==0,x//2,3*x+1)

print(next_collatz(tf.constant([1,2])))

with assert_raises(ValueError):
  next_collatz(tf.constant([[1,2],[2,3]]))

with assert_raises(ValueError):
  next_collatz(tf.constant([1.0,2.0]))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)
Caught excepted exception 
 <class 'ValueError'>
Caught excepted exception 
 <class 'ValueError'>


Traceback (most recent call last):
  File "<ipython-input-4-46ae65889eac>", line 4, in assert_raises
    yield
  File "<ipython-input-21-1e2b9d312016>", line 9, in <module>
    next_collatz(tf.constant([[1,2],[2,3]]))
ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[1 2]
 [2 3]], shape=(2, 2), dtype=int32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None))
Traceback (most recent call last):
  File "<ipython-input-4-46ae65889eac>", line 4, in assert_raises
    yield
  File "<ipython-input-21-1e2b9d312016>", line 12, in <module>
    next_collatz(tf.constant([1.0,2.0]))
ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2.], shape=(2,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None))


Specifying a [None] dimension in `tf.TensorSpec` to allow for flexibility in trace reuse

Since Tensorflow matches tensors based on their shape, using a `None` dimension as a wildcard, it will allow `Function`s to reuse traces for variabley sized input or images of different size for each batch

In [22]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None],dtype=tf.int32),))
def g(x):
  print('Tracing with ',x)
  return x

print(g(tf.constant([1,2,3])))
print(g(tf.constant([1,2,3,4,5])))

Tracing with  Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([1 2 3], shape=(3,), dtype=int32)
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int32)


* Cast Python arguments to Tensors reducing the retracing: 

Often, Python arguments are used to controll the hyperparameters and graph constructions for example 
`num_layers=10` or `training=True` or `nonlinearity=relu`. So if the Python arguments changes, it makes sense that one would have to retrace the graph

It is also possible that the argument is not used in graph construction. In these cases, a change in the Python value can trigger needless retracing. Despite the multiple traces, the generated graph is  actually identitcal thus retracing is unnecessary


In [23]:
def train_one_step():
  pass

@tf.function
def train(num_steps):
  print("Tracing with num_steps=",num_steps)
  tf.print("Executing with num_steps = ",num_steps)
  for _ in tf.range(num_steps):
    train_one_step()

print("Retracing occurs different for different python arguments")
train(num_steps=10)
train(num_steps=20)

print()
print("Traces are reused for Tensor arguments.")
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

Retracing occurs different for different python arguments
Tracing with num_steps= 10
Executing with num_steps =  10
Tracing with num_steps= 20
Executing with num_steps =  20

Traces are reused for Tensor arguments.
Tracing with num_steps= Tensor("num_steps:0", shape=(), dtype=int32)
Executing with num_steps =  10
Executing with num_steps =  20


If one needs a force retracing, the different versions of the functions will not be sharing traces

In [24]:
def f():
  print('Tracing!')
  print('Executing!!')

tf.function(f)()
tf.function(f)()  

Tracing!
Executing!!
Tracing!
Executing!!


## Python side effects: 

Python side effects like printing, appending to list and mutating globals only happens the first time one calls the function. The graph is constructed in the first run. `tf.Graph` is reexecuted, without executing the Python code.

The general rule of thumb is to only use Python side effects to debug your traces. Otherwise tf.Variable.assign, tf.print, and tf.summary are the best way for retracing during tensorflow runtime

In [25]:
@tf.function
def f(x):
  print("Traced with",x)
  tf.print("Executed with",x)
f(1)
f(1)
f(2)
f(tf.constant(1))
f(tf.constant(2))
f(tf.constant(3))
f(tf.constant(4))

Traced with 1
Executed with 1
Executed with 1
Traced with 2
Executed with 2
Traced with Tensor("x:0", shape=(), dtype=int32)
Executed with 1
Executed with 2
Executed with 3
Executed with 4


A generator might be used for keeping track of the states. This works well in eager mode execution. The same result might not work well in the function


In [26]:
external_var = tf.Variable(0)
@tf.function
def buggy_consume_next(iterator):
  external_var.assign_add(next(iterator))
  tf.print("Value of external_var:",external_var)

iterator = iter([0,1,2,3,4])
buggy_consume_next(iterator)

buggy_consume_next(iterator)
buggy_consume_next(iterator)

Value of external_var: 0
Value of external_var: 0
Value of external_var: 0


If one would like to execute a python code during invokation of a Function `tf.py_function` is an exit hatch. The drawback of `tf.py_function` is that its not portable or particularly performat, nor does it works well with distribution setups. Also since tf.py_function has to be wired into graphs, it casts all inputs/outputs to tensors

APIs like tf.gather, tf.stack and tf.TensorArray can help one to implement common looping patterns in native Tensorflow

In [27]:
external_list= []

def side_effect(x):
  print("Python side effects")
  external_list.append(x)

@tf.function
def f(x):
  tf.py_function(side_effect,inp=[x],Tout=[])

f(1)
f(1)
f(1)

assert len(external_list) == 3
assert external_list[0].numpy() == 1

Python side effects
Python side effects
Python side effects


### Variables: 

One may encounter an error when creating a new `tf.Variable` in a function. This error guards against the behavior of divergence on repeated calls. In eager mode the varaible is created on every call. In function mode the variable may not be created due to common retracing

In [28]:
@tf.function
def f(x):
  v = tf.Variable(1.0)
  v.assign_add(x)
  return v


In [29]:
with assert_raises(ValueError):
  f(1.0)

Caught excepted exception 
 <class 'ValueError'>


Traceback (most recent call last):
  File "<ipython-input-4-46ae65889eac>", line 4, in assert_raises
    yield
  File "<ipython-input-29-393b338b5a82>", line 2, in <module>
    f(1.0)
ValueError: in user code:

    <ipython-input-28-9ce361fb3439>:3 f  *
        v = tf.Variable(1.0)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/variables.py:262 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/variables.py:256 _variable_v2_call
        shape=shape)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/variables.py:67 getter
        return captured_getter(captured_previous, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py:702 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.



Variable can be created inside the function as long as those variables are only created the first time the function is executed

In [30]:
class Count(tf.Module):
  def __init__(self):
    self.count = None
  
  @tf.function
  def __call__(self):
    if self.count is None:
      self.count = tf.Variable(0)
    return self.count.assign_add(1)

c = Count()
print(c())
print(c())
print(c())
print(c())

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


Another error one might encounter is the garbage collected variable. Unlimke normal Python functions, these function only retrain WeakRefs to the variables they close over, one must retain a reference to any variables

In [31]:
external_var = tf.Variable(3)
@tf.function
def f(x):
  return x*external_var

trace_f = f.get_concrete_function(4)
print("Calling concrete function")
print(trace_f(4))

del external_var
print()
print("Calling concrete function after the garbage collection its  weakly referenced variable")
with assert_raises(tf.errors.FailedPreconditionError):
  trace_f(4)

Calling concrete function
tf.Tensor(12, shape=(), dtype=int32)

Calling concrete function after the garbage collection its  weakly referenced variable
Caught excepted exception 
 <class 'tensorflow.python.framework.errors_impl.FailedPreconditionError'>


Traceback (most recent call last):
  File "<ipython-input-4-46ae65889eac>", line 4, in assert_raises
    yield
  File "<ipython-input-31-08ec80324c16>", line 14, in <module>
    trace_f(4)
tensorflow.python.framework.errors_impl.FailedPreconditionError:  Error while reading resource variable _AnonymousVar4 from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/_AnonymousVar4/N10tensorflow3VarE does not exist.
	 [[node ReadVariableOp (defined at <ipython-input-31-08ec80324c16>:4) ]] [Op:__inference_f_979]

Function call stack:
f



# AutoGraph Transformations

Autograph is a library is on by default in `tf.function`, and transforms a subset of Python eager code into graph compatible tensorflow ops. This includes control flows like `if`,`for` and `while`

Tensorflow ops like `tf.cond` and `tf.while_loop` continue to work, but control flow easier to write and understand in Python

In [32]:
#Simple loop

@tf.function
def f(x):
  while tf.reduce_sum(x)>1:
    tf.print(x)
    x = tf.tanh(x)
  return x

f(tf.random.uniform([5]))

[0.664173365 0.969144344 0.879497886 0.0735514164 0.600046515]
[0.581134081 0.74832803 0.706167758 0.0734190643 0.537082672]
[0.523489237 0.634150445 0.608268082 0.0732874274 0.490776539]
[0.480388522 0.560903549 0.542906821 0.0731564909 0.454832554]
[0.44655472 0.508647501 0.495185167 0.0730262548 0.425863236]
[0.419062912 0.468890727 0.458322108 0.0728967115 0.401858389]
[0.396140695 0.437302619 0.428715676 0.0727678537 0.381537914]
[0.376641959 0.4114061 0.404247433 0.0726396814 0.364042312]
[0.359787643 0.389665931 0.383577347 0.0725121871 0.348769635]
[0.34502697 0.371072173 0.365810156 0.0723853558 0.335283905]
[0.331957847 0.354929149 0.350321501 0.0722591802 0.323260576]
[0.320278883 0.340739667 0.336660624 0.0721336678 0.312452137]
[0.309759051 0.328137577 0.324492872 0.0720088184 0.302666247]
[0.300217867 0.316846281 0.313563734 0.0718846098 0.293750703]
[0.291511953 0.306652516 0.303675652 0.0717610344 0.285583287]
[0.283525795 0.297388703 0.294672728 0.0716381 0.278064609]


<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.2285162 , 0.2355673 , 0.23421493, 0.07044253, 0.22563568],
      dtype=float32)>

In [33]:
print(tf.autograph.to_code(f.python_function))

def tf__f(x):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body():
            nonlocal x
            ag__.converted_call(ag__.ld(tf).print, (ag__.ld(x),), None, fscope)
            x = ag__.converted_call(ag__.ld(tf).tanh, (ag__.ld(x),), None, fscope)

        def loop_test():
            return (ag__.converted_call(ag__.ld(tf).reduce_sum, (ag__.ld(x),), None, fscope) > 1)
        ag__.while_stmt(loop_test, loop_body, get_state, set_state, ('x',), {})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



## Conditionals:

Autograph will convert some `if<condition>` statement into its equivalent `tf.cond` calls. This substitution is made if `<condition>` is a Tensor. Otherwise `if` statement is executed as a Python conditional

A Python conditional executes during tracing, so exactly one branch of the conditional will be aded to the graph. Without AutoGraph, this traced graph would be unable to take the alternative branch if there is data-dependent control flow. 

`tf.cond` traces and adds both branches of the conditional to the graph, dynamically selecting a branch at execution time. Tracing can have unintended side effects

In [34]:
@tf.function
def fizzbuzz(n):
  for i in tf.range(1, n+1):
    print('Tracing for loop')
    if i%15==0:
      print('Tracing fizzbuzz branch')
      tf.print('fizzbuzz')
    elif i%3 ==0:
      print('Tracing fizz branch')
      tf.print('fizz')
    elif i%5 ==0:
      print('Tracing buzz branch')
      tf.print('buzz')
    else:
      print('Testing default branch')
      tf.print(i)

fizzbuzz(tf.constant(5))
fizzbuzz(tf.constant(30))


Tracing for loop
Tracing fizzbuzz branch
Tracing fizz branch
Tracing buzz branch
Testing default branch
1
2
fizz
4
buzz
1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz
16
17
fizz
19
buzz
fizz
22
23
fizz
buzz
26
fizz
28
29
fizzbuzz


### Loops:

Autograph will convert some `for` and `while` statements into the equivlen Tensorflow loops ops, like tf.while_loop. If not converted, the for or while loop is executed as a Python loop. 

This substitution is made in the following situations

* `for x in y`: if `y` is a Tensor, convert to `tf.while_loop`. In the special case where `y` is a `tf.data.Dataset`, combination of `tf.data.Dataset` ops are generated
* while `<condition>`: if `<condition>` is a Tensor, convert to `tf.while_loop`

A python loop executes during tracing and adding the additional ops to the `tf.Graph`. for every iteration of the loop. 

A Tensorflow loop traces the body of the loop and dynamically selects how many Iterations to run at execution time. The loop body only executes once in `tf.Graph`


A common pitfall is looping over python/numpy data in function `tf.function`. This loop will run during the plotting process, adding a copy of your model to `tf.Graph` for each iteration

If we want the `tf.function` whole training loop in `tf.function`,the safest way to do that is to have `tf.data.Dataset` so that AutoGraph dunamically unwinds the training loop.


In [40]:
def measure_graph_size(f,*args):
  g = f.get_concrete_function(*args).graph
  print("{} {} contains {} nodes in its graph".format(f.__name__,', '.join(map(str,args)),len(g.as_graph_def().node)))

@tf.function
def train(dataset):
  loss = tf.constant(0)
  for x,y in dataset:
    loss+= tf.abs(y-x)
  return loss

small_data = [(1,1)]*3
big_data = [(1,1)]*10
measure_graph_size(train,small_data)
measure_graph_size(train,big_data)

measure_graph_size(train,tf.data.Dataset.from_generator(lambda:small_data,(tf.int32,tf.int32)))
measure_graph_size(train,tf.data.Dataset.from_generator(lambda:big_data,(tf.int32,tf.int32)))


train [(1, 1), (1, 1), (1, 1)] contains 11 nodes in its graph
train [(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)] contains 32 nodes in its graph
train <FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)> contains 8 nodes in its graph
train <FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)> contains 8 nodes in its graph


When we wrap Python/Numpy data in a dataset, relative to. The former, will keep the data in Python and retrieve it via which may have performance implications, while the latter will aggregate a copy of the data as a large node in the graph, which may have implication for performance on memory. `tf.data.Dataset.from_generator`
`tf.data.Dataset.from_tensors`

Reading data from files via TFRecordDataset/CsvDataset/ etc is the most efficient way to consume the data and does not involve data loading or prefeteching using Python. 

## Accumulating values in a loop

In [41]:
batch_size =2
seq_len  = 3
feature_size = 4

def rnn_step(inp,state):
  return inp + state

@tf.function
def dynamic_rnn(rnn_step,input_data,initial_state):
  input_data = tf.transpose(input_data,[1,0,2])
  max_seq_len = input_data.shape[0]

  states = tf.TensorArray(tf.float32,size=max_seq_len)
  state = initial_state
  for i in tf.range(max_seq_len):
    state = rnn_step(input_data[i],state)
    states = states.write(i,state)
  return tf.transpose(states.stack(),[1,0,2])

dynamic_rnn(rnn_step,tf.random.uniform([batch_size,seq_len,feature_size]),
            tf.zeros([batch_size,feature_size])
            )



<tf.Tensor: shape=(2, 3, 4), dtype=float32, numpy=
array([[[0.7389803 , 0.25470185, 0.61482334, 0.99299073],
        [1.2608832 , 0.56535447, 0.7802501 , 1.2835064 ],
        [1.7465137 , 0.71508026, 0.9849535 , 1.5519809 ]],

       [[0.9971254 , 0.1277901 , 0.17888403, 0.94876456],
        [1.3096862 , 0.40042984, 0.7757361 , 1.0846279 ],
        [2.244003  , 1.0542338 , 1.3096174 , 1.1859447 ]]], dtype=float32)>